# Instalamos la libreria pytesseract

In [ ]:
%pip install pytesseract


# Ubicar la imagen de ejemplo de recibo


![Texto crudo del recipe medico](./app/static/img/text2.jpg)


In [ ]:
import pytesseract

In [ ]:
custom_config = r'--oem 3 --psm 6 --user-words "spa.user-words"'
raw_text = pytesseract.image_to_string("/code/app/static/img/text2.jpg" , config=custom_config , lang='spa')
print(raw_text)

In [ ]:
list_words = raw_text.lower().split("\n")
print(list_words)

In [ ]:
items_filtered = [item for item in list_words if len(item) >= 8]
print(items_filtered)

In [ ]:
#el texto crudo debe limpiarse. para eso metemos esta lib
%pip install clean-text


In [ ]:
from cleantext import clean
cleaned_text = clean(raw_text, fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="<CUR>",
    lang="en")
print(cleaned_text.split("cant")[-1].split("\n"))


In [ ]:
# Initializing string 
import string
 
# Using list comprehension and isalnum() method to extract words from string
res = [word.strip(string.punctuation) for word in cleaned_text.split() if word.strip(string.punctuation).isalnum() and len(word) > 9]
# Printing result
print(list(set(res)))

In [ ]:
# remove duplicated words
res= list(set(res))
res

In [6]:
# Of current list only extract words of dicctionary
def read_glosary_and_return_list(route_glosary = 'code/app/spa.user-words'):
    with open(route_glosary) as f:
        glosary = f.read()
    return glosary.split("\n")

glosary = read_glosary_and_return_list()

In [ ]:
#Matches in glosary

# The second parameter is the input iterable
# The filter() applies the lambda to the iterable
# and only returns all matches where the lambda evaluates
# to true
def return_word_if_matches_on_glosary(word):
    return list(filter(lambda a: word in a, glosary))[-1] #return last match

return_word_if_matches_on_glosary("azitromicina")

In [20]:
import pytesseract
from cleantext import clean
import string

class RecipeProcessor:
    def __init__(self, route_glossary='/code/app/spa.user-words'):
        self.route_glossary = route_glossary
        self.glossary_words = self._read_glossary_and_return_list()

    def _read_glossary_and_return_list(self):
        with open(self.route_glossary) as f:
            glossary = f.read()
        return glossary.split("\n")

    def extract_raw_text_from_recipe(self, route_img_recipe="code/app/static/uploads/test.png"):
        custom_config = r'--oem 3 --psm 6 --user-words "spa.user-words"'
        raw_text = pytesseract.image_to_string(
            route_img_recipe, lang='spa', config=custom_config)
        return raw_text

    def _clean_raw_text(self, raw_text):
        return clean(raw_text, fix_unicode=True, to_ascii=True, lower=True)

    def extract_words_from_cleaned_text_and_return_list(self, cleaned_text):
        return [word.strip(string.punctuation) for word in cleaned_text.split() if word.strip(string.punctuation).isalnum() and len(word) > 9]

    def _remove_duplicates_on_list(self, list_words):
        return list(set(list_words))

    def return_glossary_words(self, list_no_duplicated_items):
        result = [word for word in list_no_duplicated_items if word in self.glossary_words]
        return result

    def process_recipe(self, route_uploaded_img_recipe="test.png"):
        raw_text = self.extract_raw_text_from_recipe( )
        cleaned_text = self._clean_raw_text(raw_text)
        words_extracted = self.extract_words_from_cleaned_text_and_return_list(cleaned_text)
        list_no_duplicated_items = self._remove_duplicates_on_list(words_extracted)
        only_glossary_words = self.return_glossary_words(list_no_duplicated_items)
        return { 'kywords_extracted': list_no_duplicated_items, 'glosary_kywords': only_glossary_words }


ocr = RecipeProcessor(route_glossary="code/app/spa.user-words")
raw_text = ocr.process_recipe()
raw_text

{'a': ['tratamiento',
  'anticoeilante',
  'corrticoide',
  'fibromialgia',
  'aitrombotio',
  'antiembolico'],
 'b': ['antiembolico']}